### Import credentials and custom notebooks

In [0]:
%run "/Users/vinhquoc2049@gmail.com/get_credentials"

In [0]:
%run "/Users/vinhquoc2049@gmail.com/bds_streaming_pipeline"

### Mount ADLS to Databricks Filesystem

In [0]:
def mount_container_adls2(container,mount_folder,sa_name,key):
    mounted_paths = [mount.mountPoint for mount in dbutils.fs.mounts()]
    if(mount_folder in mounted_paths):
        print(f"The mount folder already '{mount_folder}' exists.")
        return False
    else:
        url = f"wasbs://{container}@{sa_name}.blob.core.windows.net/"
        conf = {
            "fs.azure.account.key.{0}.blob.core.windows.net".format(sa_name): account_key
        }
        # Mount the Azure Data Lake Storage Gen2
        dbutils.fs.mount(
            source=url,
            mount_point=mount_folder,
            extra_configs=conf
        )
        print(f"Mount container {container} in {sa_name} into Databrick successfully")
        return True

In [0]:
# Replace the placeholders with your actual values
sa_name = spark.conf.get("sa_name")
connection_string = spark.conf.get("connection_string")
key = spark.conf.get("key")

container = "bronze"
mount_folder = "/mnt/bronze"
mount_container_adls2(container,mount_folder,sa_name,key)

container = "silver"
mount_folder = "/mnt/silver"
mount_container_adls2(container,mount_folder,sa_name,key)

The mount folder already '/mnt/bronze' exists.
The mount folder already '/mnt/silver' exists.
Out[43]: False

### Connect to ADLS and run the data pipeline

In [0]:
from delta.tables import DeltaTable
import os
from time import sleep

def stop_all_streaming_queries():
    # Get the active streaming queries
    active_queries = spark.streams.active

    # Check if there are any active queries
    if active_queries:
        print("List of Active Streaming Queries:")
        for query in active_queries:
            print(f"Query Name: {query.name}, ID: {query.id}")
            query.stop()
    else:
        print("No active streaming queries.")


def is_delta_table(table_path):
    try:
        delta_table = DeltaTable.forPath(spark, table_path)
        return delta_table is not None
    except Exception as e:
        return False

def main():
    bronze_base_dir='/mnt/bronze'
    bronze_read_path='/mnt/bronze/scraped_data'


    silver_base_dir='/mnt/silver'
    silver_read_path=bronze_base_dir


    bronze=BronzeBDS(bronze_read_path,bronze_base_dir,bronze_base_dir)
    silver=SilverBDS(bronze_base_dir,silver_base_dir,silver_base_dir)

    # Trigger as batch pipeline, change this parameter if you want stream pipeline
    trigger='batch'
    bzQuery=bronze.process(trigger)
    bzQuery.awaitTermination()
    slQuery=silver.process(trigger)


stop_all_streaming_queries()
main()

No active streaming queries.

Starting Bronze Stream...

Starting Silver Stream...